<a href="https://colab.research.google.com/github/byerHu/Machine-learning-algorithm-source-code-implementation/blob/master/knn%E6%BA%90%E7%A0%81%E5%AE%9E%E7%8E%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from math import sqrt
from collections import Counter

In [0]:
def KNN_classify(k,X_train,y_train,x):
  """
    k:表示knn的中k的值
    X_train: 训练集的features
    y_train: 训练集的labels
    x: 新的数据
  """
  assert 1<=k<=X_train.shape[0],"k must be valid"
  assert X_train.shape[0] == y_train.shape[0], "the size of X_train must equal to the size of y_train"
  assert X_train.shape[1] == x.shape[0], "the feature number of x must to be equal to X_train"
  
  distances = [sqrt(np.sum((x_train-x)**2)) for x_train in X_train]
  nearest = np.argsort(distances)
  
  topK_y = [y_train[i] for i in nearest]
  votes = Counter(topK_y)
  
  return votes.most_common(1)[0][0]

In [0]:
x = [[0,0],
     [1,1],
     [2,2],
     [10,10],
     [11,11],
     [12,12]]
y = [0,0,0,1,1,1]

X_train = np.array(x)
y_train = np.array(y)
print(X_train.shape)
x = np.array([13,13])
KNN_classify(2,X_train,y_train,x)

(6, 2)


1